In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


In [3]:
df = pd.read_excel('MUNIC//Base_MUNIC_2018.xlsx', sheet_name='Assistência social', usecols=['Cod Municipio','MASS01','MASS03','MASS04','MASS05','MASS06'])
df

,Cod Municipio,MASS01,MASS03,MASS04,MASS05,MASS06
0,1100015,Secretaria em conjunto com outras políticas se...,Masculino,46,Parda,Ensino fundamental (1º Grau) incompleto
1,1100023,Secretaria exclusiva,Feminino,34,Preta,Especialização
2,1100031,Secretaria exclusiva,Feminino,54,Parda,Ensino superior completo
3,1100049,Secretaria exclusiva,Masculino,51,Parda,Ensino superior completo
4,1100056,Secretaria exclusiva,Feminino,58,Parda,Especialização
...,...,...,...,...,...,...
5565,5222005,Secretaria exclusiva,Feminino,40,Branca,Especialização
5566,5222054,Secretaria em conjunto com outras políticas se...,Feminino,52,Branca,Ensino superior completo
5567,5222203,Secretaria exclusiva,Feminino,68,Parda,Ensino superior completo
5568,5222302,Secretaria em conjunto com outras políticas se...,Feminino,42,Parda,Ensino médio (2º Grau) completo


In [4]:
uf = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name = 'Variáveis externas', usecols=[0,1,2,3])
uf

,CodMun,UF,Cod UF,Mun
0,1100015,RO,11,Alta Floresta DOeste
1,1100023,RO,11,Ariquemes
2,1100031,RO,11,Cabixi
3,1100049,RO,11,Cacoal
4,1100056,RO,11,Cerejeiras
...,...,...,...,...
5565,5222005,GO,52,Vianópolis
5566,5222054,GO,52,Vicentinópolis
5567,5222203,GO,52,Vila Boa
5568,5222302,GO,52,Vila Propício


In [5]:
df= df.rename(columns={'Cod Municipio':'id_municipio',
                        'MASS01':'caracterizacao_orgao_gestor',
                        'MASS03':'genero',
                        'MASS04' :'idade',
                        'MASS05':'cor_raca',
                        'MASS06':'grau_instrucao'}) 

In [6]:
x= uf.pivot_table(columns=('CodMun', 'UF','Cod UF', 'Mun'), aggfunc='size')


In [7]:
uf = pd.DataFrame(x).reset_index()[['CodMun', 'UF','Cod UF', 'Mun']]

In [8]:
df = df.merge(uf, right_on='CodMun',left_on='id_municipio') #adicionando siglas e nome das UFs e dos municipios


In [9]:
df['ano']=2018

In [10]:
df = df.drop(['CodMun'], axis=1)

In [11]:
df = df.rename(columns={'UF':'sigla_uf',
                        'Cod UF':'cod_uf',
                        'Mun':'nome_municipio'}) #padronizando as colunas

In [12]:
df['caracterizacao_orgao_gestor']=df['caracterizacao_orgao_gestor'].str.title()

In [13]:
#padronizando os dados da coluna 'genero'
df['genero']=np.where(df['genero']=='(**) Sem gestor','Sem titular',df['genero'])
df['genero']=np.where(df['genero']=='-','Sem dados',df['genero'])
df['genero']=np.where(df['genero']=='(*) Não soube informar','Sem dados',df['genero'])

#padronizando os dados da coluna 'idade'
df['idade']=np.where(df['idade']=='(*) Não soube informar',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='-',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='(**) Sem gestor',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Recusa',np.nan,df['idade'])


#padronizando os dados da coluna 'cor_raca'

df['cor_raca']=np.where(df['cor_raca']=='(**) Sem gestor','Sem titular',df['cor_raca'])
df['cor_raca']=np.where(df['cor_raca']=='-','Sem dados',df['cor_raca'])

#padronizando os dados da coluna 'escolaridade'
df['grau_instrucao']=np.where(df['grau_instrucao']=='-','Sem dados',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='(**) Sem gestor','Sem titular',df['grau_instrucao'])


In [14]:
df['idade'] =pd.to_numeric(df['idade']) #transformando em numerico 


In [15]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [16]:
df=df[['ano', 'sigla_uf','nome_municipio','id_municipio','caracterizacao_orgao_gestor', 'genero', 'cor_raca', 'grau_instrucao', 'faixa_etaria']] #reorganizando as colunas

In [17]:
df['grau_instrucao'].unique()

array(['Ensino fundamental (1º Grau) incompleto', 'Especialização',
       'Ensino superior completo', 'Ensino médio (2º Grau) completo',
       'Mestrado', 'Ensino superior incompleto',
       'Ensino médio (2º Grau) incompleto',
       'Ensino fundamental ( 1º Grau) completo', 'Recusa', 'Doutorado',
       'Sem dados', 'Sem titular'], dtype=object)

In [18]:
# criando dicionário
dict_esco = {'Ensino fundamental (1º Grau) incompleto':'Até Ensino Fundamental',
             'Ensino fundamental ( 1º Grau) completo':'Até Ensino Fundamental',
             'Ensino médio (2º Grau) incompleto':'Até Ensino Médio',
             'Ensino médio (2º Grau) completo':'Até Ensino Médio',
             'Ensino superior incompleto':'Até Ensino Superior Completo',
             'Ensino superior completo':'Até Ensino Superior Completo',
             'Especialização':'Até Pós Graduação ou Mestrado',
             'Mestrado':'Até Pós Graduação ou Mestrado',
             'Doutorado':'Até Doutorado'}


In [19]:
df = df.replace({'grau_instrucao':dict_esco}) #substituindo valores para padronizar


In [21]:
df.columns

Index(['ano', 'sigla_uf', 'nome_municipio', 'id_municipio',
       'caracterizacao_orgao_gestor', 'genero', 'cor_raca', 'grau_instrucao',
       'faixa_etaria'],
      dtype='object')

Subindo para o GBQ

In [ ]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   ano                          5570 non-null   int64   
 1   sigla_uf                     5570 non-null   object  
 2   nome_municipio               5570 non-null   object  
 3   id_municipio                 5570 non-null   int64   
 4   caracterizacao_orgao_gestor  5570 non-null   object  
 5   genero                       5570 non-null   object  
 6   cor_raca                     5570 non-null   object  
 7   grau_instrucao               5570 non-null   object  
 8   faixa_etaria                 5515 non-null   category
dtypes: category(1), int64(2), object(6)
memory usage: 397.3+ KB


In [ ]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação referente municipio'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Código do IBGE do município'),
        bigquery.SchemaField('caracterizacao_orgao_gestor','STRING',description='Tipo de órgão no qual o gestor está'),
        bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
        bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação')
        ]


In [ ]:
table_ref = dataset_ref.table('MUNIC_perfil_gestor_assistencia_social_tipo_orgao')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=19dc96a4-a165-4c96-a959-e1a8cd998bbd>